# MA8701 Prject II:

https://github.com/metno/havvarsel-data-driven-pred/blob/main/dataset1.csv

God morgen! As promised here is our data set 🥳 I wanted to include one more covariate (a oceanographic one) but I failed during the weekend - I will try it after the lecture again. BUT as soon as you start with data analysis please let know since I dont want to interrupt ongoing work with changing the data set afterwards!

As discussed the data set consists of the following timeseries:
- water_temp (the response)
- measurements of 7 atmospheric values... the naming convention is "station_id"(where the measurement is done) + "value"(what got measured) + "number"(0=measurement in 2m height, 1=measurement in 10 height above ground)

We have
- water_temp in degC - measured at 7,10,14, 16 every day during swimming season

- air_temperature in degC
- wind_speed in m/s
- cloud_area_fraction in 0,...,8 (0 no clouds, 8 fully cloudy)
- mean(solar irradiance) in w/m2 - a quantity for the intensity of the sun
- sum(duration of sunshine) in min - sunshine in the last hour
- mean(realtive humidity) in %
- mean(downwelling shortwave flux) in W/m2 - like irradiance but only for UV radiation (normally criterion for getting sun burned, but I thought that it could maybe get interesting)

I hope thats something we can work with :) In case of questions just ask me since I was fighting a lot with those weather data bases and know somehow what is going on with weather measurements in Norway (not forecasts though haha)

In [105]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.5.0-rc1


In [106]:
# Import data
column_names = ['water_temp', 'air_temp1', 'air_temp2', 'air_temp3', 'air_temp4',
                'wind_speed1', 'wind_speed2', 'wind_speed3', 'cloud1', 'cloud2',
                'cloud3', 'cloud4', 'solar_mean', 'solar_sum', 'humidity1', 'humidity2',
                'surface_downwelling']
# raw_dataset = pd.read_csv('dataset1.csv', names = column_names, header = None, index_col = 'time')
raw_dataset = pd.read_csv('dataset1.csv', header = None, names = column_names)
raw_dataset = raw_dataset.iloc[1:, :]


In [107]:

shifted_water_temp = raw_dataset['water_temp']
shifted_water_temp = shifted_water_temp.shift(1)
new_dataset = raw_dataset
new_dataset['new_water_temp'] = shifted_water_temp
new_dataset = new_dataset.iloc[1:, :]
# print(raw_dataset.iloc[:, 0]>'2020-05-15 07:00:00+00:00')


In [108]:
dataset = new_dataset.copy()
# print(dataset.isna().sum())
del dataset["wind_speed3"] # remove empty data
del dataset["cloud2"]
dataset.tail()
for i in range(len(dataset.columns)):
    dataset.iloc[:,i] = pd.to_numeric(dataset.iloc[:,i], downcast = "float")
dataset


,water_temp,air_temp1,air_temp2,air_temp3,air_temp4,wind_speed1,wind_speed2,cloud1,cloud3,cloud4,solar_mean,solar_sum,humidity1,humidity2,surface_downwelling,new_water_temp
2016-05-24 10:00:00+00:00,12.0,9.800000,9.0,11.700000,11.100000,3.6,1.4,8.0,8.0,5.0,0.737,0.0,83.0,76.0,200.899994,11.7
2016-05-24 14:00:00+00:00,12.1,13.000000,12.7,14.500000,14.200000,4.2,1.4,7.0,8.0,5.0,0.737,0.0,66.0,78.0,330.000000,12.0
2016-05-24 16:00:00+00:00,12.3,12.400000,12.5,14.500000,13.900000,5.0,1.5,7.0,8.0,3.0,0.737,1.0,64.0,69.0,281.600006,12.1
2016-05-25 07:00:00+00:00,11.6,11.900000,10.0,13.100000,11.600000,3.4,1.8,1.0,8.0,4.0,0.737,0.0,59.0,55.0,392.799988,12.3
2016-05-25 11:00:00+00:00,12.6,15.400000,15.0,17.200001,16.900000,3.9,1.4,1.0,8.0,1.0,0.737,60.0,48.0,48.0,801.000000,11.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-14 08:00:00+00:00,15.6,16.100000,17.6,14.600000,16.600000,2.3,0.5,5.0,0.0,2.0,0.148,44.0,90.0,69.0,249.899994,15.8
2020-09-14 10:00:00+00:00,15.7,19.400000,20.1,17.500000,19.500000,3.2,2.2,6.0,3.0,5.0,0.630,6.0,77.0,74.0,245.500000,15.6
2020-09-14 13:00:00+00:00,16.0,19.700001,21.4,20.500000,21.299999,3.7,2.9,8.0,1.0,5.0,0.777,60.0,69.0,77.0,489.000000,15.7
2020-09-14 16:00:00+00:00,15.8,21.000000,20.4,21.400000,20.900000,1.5,1.1,8.0,0.0,5.0,0.535,12.0,72.0,75.0,169.800003,16.0


In [109]:
# Split the data set
train_dataset = dataset.iloc[:1856, :]
test_dataset = dataset.drop(train_dataset.index)
print(train_dataset.tail())
print(test_dataset.head())

                           water_temp  air_temp1  air_temp2  air_temp3  \
2019-10-01 07:00:00+00:00        13.2        7.1        6.9        9.0   
2019-10-01 10:00:00+00:00        13.8       11.5       10.2       12.6   
2019-10-01 13:00:00+00:00        13.5       13.4       12.0       14.5   
2019-10-01 16:00:00+00:00        13.6       10.9        9.9       13.9   
2019-10-02 07:00:00+00:00        12.2        7.0        5.8        9.6   

                           air_temp4  wind_speed1  wind_speed2  cloud1  \
2019-10-01 07:00:00+00:00        7.5          1.6          0.5     0.0   
2019-10-01 10:00:00+00:00       13.5          2.1          0.8     4.0   
2019-10-01 13:00:00+00:00       14.7          1.5          1.0     1.0   
2019-10-01 16:00:00+00:00       10.9          0.7          0.4     7.0   
2019-10-02 07:00:00+00:00        7.1          4.7          1.0     8.0   

                           cloud3  cloud4  solar_mean  solar_sum  humidity1  \
2019-10-01 07:00:00+00:00     0

In [ ]:
sns.pairplot(train_dataset, vars = train_dataset.columns[:], diag_kind='kde')
plt.show()

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('new_water_temp')
test_labels = test_features.pop('new_water_temp')

train_dataset.describe().transpose()

In [ ]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [water_temp]')
  plt.legend()
  plt.grid(True)

In [ ]:
def plot_air_temp1(x, y):
  plt.scatter(train_features['air_temp1'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Air_temp1')
  plt.ylabel('water_temp')
  plt.legend()

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

linear_model.predict(train_features)
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

history = linear_model.fit(
    train_features, train_labels,
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

plot_loss(history)


In [ ]:
test_results = linear_model.evaluate(
    test_features, test_labels, verbose=0)
print(test_results)
test_predictions = linear_model.predict(test_features).flatten()

# plot the estimation
x = np.arange(len(test_labels))
plt.scatter(x, test_labels)
plt.scatter(x, test_predictions)
plt.show()

# Conclusions from Deep learning part

Air temperature measurements seem having high correlations, so do the humidity. By using single layer and multiple layers neural
network, it predicts the water temperature with relatively low error.

